### Initial Values
First, we create variables and generate important information for our calculations.

In [1]:
import pandas as pd
# define options for suits
suits = ['H','D','S','C']
# define options for values
# Aces are notated as 1s, Jacks are notated as 11s, Queens are notated as 12s, Kings are notated as 13s
values = list(range(1,14))
# use list comprehensions to get a data frame of all possible cards
# each suit repeated 13 times; list of values repeated 4 times
cards_df = pd.DataFrame(data = {'suit':[y for z in [13*[x] for x in suits] for y in z],'value':values*4})
print(cards_df.head(3)) # print first 3 rows
print(cards_df.tail(3)) # print last 3 rows

  suit  value
0    H      1
1    H      2
2    H      3
   suit  value
49    C     11
50    C     12
51    C     13


In [2]:
# create empty list to store all possible cards
cards_list = []
# for each row in the data frame, define the card as a list of [suit,value]
# then append the card to the list
for i in range(0,len(cards_df)):
    card = [cards_df.iloc[i].suit,cards_df.iloc[i].value]
    cards_list.append(card)
print(cards_list) # show the resulting list

[['H', 1], ['H', 2], ['H', 3], ['H', 4], ['H', 5], ['H', 6], ['H', 7], ['H', 8], ['H', 9], ['H', 10], ['H', 11], ['H', 12], ['H', 13], ['D', 1], ['D', 2], ['D', 3], ['D', 4], ['D', 5], ['D', 6], ['D', 7], ['D', 8], ['D', 9], ['D', 10], ['D', 11], ['D', 12], ['D', 13], ['S', 1], ['S', 2], ['S', 3], ['S', 4], ['S', 5], ['S', 6], ['S', 7], ['S', 8], ['S', 9], ['S', 10], ['S', 11], ['S', 12], ['S', 13], ['C', 1], ['C', 2], ['C', 3], ['C', 4], ['C', 5], ['C', 6], ['C', 7], ['C', 8], ['C', 9], ['C', 10], ['C', 11], ['C', 12], ['C', 13]]


In [3]:
import itertools
# make a list of all possible 5 card combinations
hands_list = list(itertools.combinations(cards_list,5))
print(hands_list[:3]) # display first 3
print(hands_list[len(hands_list)-3:]) # display last 3

[(['H', 1], ['H', 2], ['H', 3], ['H', 4], ['H', 5]), (['H', 1], ['H', 2], ['H', 3], ['H', 4], ['H', 6]), (['H', 1], ['H', 2], ['H', 3], ['H', 4], ['H', 7])]
[(['C', 8], ['C', 9], ['C', 11], ['C', 12], ['C', 13]), (['C', 8], ['C', 10], ['C', 11], ['C', 12], ['C', 13]), (['C', 9], ['C', 10], ['C', 11], ['C', 12], ['C', 13])]


## Functions
Next, we create functions that will take a given poker hand and generate the information we need.

In [4]:
# create a user-defined function (UDF) to convert a 5 card hand into the "category" of hand
def hand_category(hand):
    # get a list of the suits in the hand
    hand_suits = [x[0] for x in hand]
    # create a list of number of each suit in the hand [hearts,diamonds,spades,clubs]
    suit_counts = [hand_suits.count(x) for x in suits]
    # get a list of the values in the hand
    hand_values = [x[1] for x in hand]
    # sort the values numerically, ascending
    hand_values.sort()
    # create a list of the number of each value in the hand [1s,2s,3s,...,11s,12s,13s]
    value_counts = [hand_values.count(x) for x in values]
    # create a list of each increase in value [value1-value0,value2-value1,value3-value2,value4-value3]
    # note that this makes a four-item list from the five-item list of values
    value_inc = [hand_values[i]-hand_values[i-1] for i in range(1,5)]
    # define the conditions for each type of hand, working from the best hand type down
    if (sum([x==1 for x in value_inc])==4 or hand_values == [10,11,12,13,1]) and max(suit_counts) == 5:
        category = "straight-flush"
    elif max(value_counts) == 4:
        category = "four-of-a-kind"
    elif sum([x==3 for x in value_counts]) == 1 and sum([x==2 for x in value_counts]) == 1:
        category = "full-house"
    elif max(suit_counts) == 5:
        category = "flush"
    elif sum([x==1 for x in value_inc])==4 or hand_values == [10,11,12,13,1]:
        category = "straight"
    elif max(value_counts) == 3:
        category = "three-of-a-kind"
    elif sum([x==2 for x in value_counts]) == 2:
        category = "two-pair"
    elif max(value_counts) == 2:
        category = "pair"
    else:
        category = "high-card"
    return category
    # the udf will return the category evaluated through the nested if-elif-else loop

In [5]:
# rank the hands based on category
def hand_rank(hand):
    category = hand_category(hand)
    if category == "straight-flush":
        rank = 1
    elif category == "four-of-a-kind":
        rank = 2
    elif category == "full-house":
        rank = 3
    elif category == "flush":
        rank = 4
    elif category == "straight":
        rank = 5
    elif category == "three-of-a-kind":
        rank = 6
    elif category == "two-pair":
        rank = 7
    elif category == "pair":
        rank = 8
    elif category == "high-card":
        rank = 9
    else:
        rank = None
    return rank

In [6]:
# create sub-ranks to distinguish which hands are better/worse within a category
def hand_sub_rank(hand): 
    # get the overall rank of the hand, corresponding to its category (pair, flush, three-of-a-kind, etc.)
    rank = hand_rank(hand)
    # get a list of the suits in the hand
    hand_suits = [x[0] for x in hand]
    # create a list of number of each suit in the hand [hearts,diamonds,spades,clubs]
    suit_counts = [hand_suits.count(x) for x in suits]
    # get a list of the values in the hand
    hand_values = [x[1] for x in hand]
    # sort the values numerically, ascending
    hand_values.sort()
    # create a list of the number of each value in the hand [1s,2s,3s,...,11s,12s,13s]
    value_counts = [hand_values.count(x) for x in values]
    # create a reversed list [13s,12s,11s,...,3s,2s,1s]
    rev_value_counts = list(reversed(value_counts))
    
    if rank == 9: # high card
        if value_counts.index(1) == 0:
            sub_rank = 1
        else:
            sub_rank = rev_value_counts.index(1) + 2
    if rank == 8: # pair
        if value_counts.index(2) == 0:
            sub_rank = 1
        else:
            sub_rank = rev_value_counts.index(2) + 2
    if rank == 7: # two-pair
        if value_counts.index(2) == 0:
            sub_rank1 = 1
            sub_rank2 = rev_value_counts.index(2) + 1
        else:
            sub_rank1 = rev_value_counts.index(2) + 2
            sub_rank2 = rev_value_counts[sub_rank1-1:].index(2) + 1
        sub_rank = sub_rank1 + (sub_rank2/100)
    if rank == 6: # three-of-a-kind
        if value_counts.index(3) == 0:
            sub_rank = 1
        else:
            sub_rank = rev_value_counts.index(3) + 2
    if rank == 5: # straight
        if value_counts.index(1) == 0 and rev_value_counts.index(1) == 0:
            sub_rank = 1
        else:
            sub_rank = rev_value_counts.index(1) + 2
    if rank == 4: # flush
        if value_counts.index(1) == 0:
            sub_rank = 1
        else:
            sub_rank = rev_value_counts.index(1) + 2
    if rank == 3: # full house
        if value_counts.index(3) == 0:
            sub_rank1 = 1
        else:
            sub_rank1 = rev_value_counts.index(3) + 2
        if value_counts.index(2) == 0:
            sub_rank2 = 1
        else:
            sub_rank2 = rev_value_counts.index(2) + 2
        sub_rank = sub_rank1 + (sub_rank2)/100
    if rank == 2: # four-of-a-kind
        if value_counts.index(4) == 0:
            sub_rank = 1
        else:
            sub_rank = rev_value_counts.index(4) + 2
    if rank == 1: # straight-flush
        if value_counts.index(1) == 0 and rev_value_counts.index(1) == 0:
            sub_rank = 1
        else:
            sub_rank = rev_value_counts.index(1) + 2
            
    return sub_rank

In [7]:
# combine hand_rank and hand_sub rank to give a single comprehensive number for ranking hands
def hand_full_rank(hand):
    hand_full_rank = hand_rank(hand) + (hand_sub_rank(hand)/100)
    return hand_full_rank

## Simulate Probabilities
We want to be able to determine a player's odds of winning with a given hand. To do this, we run a number of simulations where opponent hands are generated at random, then the game's result (win/tie/loss) is determined. The overall proportion of simulated wins over total trials gives us a point estimate for the probability that the player will win.

In [8]:
import random
# create a function to simulate odds for an inputted hand and a given number simulation trials, with a given number of opponents.
def simulate_match(hand, opponents, trials):
    win_count = 0
    tie_count = 0
    loss_count = 0
    player_rank = hand_full_rank(hand)
    for i in range(trials):
        cards_remaining = cards_list.copy()
        for card in hand:
            cards_remaining.remove(card)
        hands_remaining = list(itertools.combinations(cards_remaining,5))
        opponent_ranks = []
        for j in range(opponents):
            opponent_hand = random.choice(hands_remaining)
            for card in opponent_hand:
                cards_remaining.remove(card)
            hands_remaining = list(itertools.combinations(cards_remaining,5))
            opponent_rank = hand_full_rank(opponent_hand)
            opponent_ranks.append(opponent_rank)
        best_opp = min(opponent_ranks)
        if player_rank < best_opp:
            win_count += 1
        if player_rank == best_opp:
            tie_count += 1
        if player_rank > best_opp:
            loss_count += 1
    win_prob = win_count / trials
    tie_prob = tie_count / trials
    loss_prob = loss_count / trials
    return win_prob, tie_prob, loss_prob

The function is set up to take inputs.

In [9]:
s1 = input('Input the suit of card 1 (H, D, S, or C)')
v1 = input('Input the value of card 1 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King)')
c1 = [str(s1),int(v1)]

s2 = input('Input the suit of card 2 (H, D, S, or C)')
v2 = input('Input the value of card 2 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King)')
c2 = [str(s2),int(v2)]

s3 = input('Input the suit of card 3 (H, D, S, or C)')
v3 = input('Input the value of card 3 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King)')
c3 = [str(s3),int(v3)]

s4 = input('Input the suit of card 4 (H, D, S, or C)')
v4 = input('Input the value of card 4 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King)')
c4 = [str(s4),int(v4)]

s5 = input('Input the suit of card 5 (H, D, S, or C)')
v5 = input('Input the value of card 5 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King)')
c5 = [str(s5),int(v5)]

trials = input('How many trials would you like to run?')
trials = int(trials)

opponents = input('How many opponents do you have?')
opponents = int(opponents)

input_hand = [c1,c2,c3,c4,c5]
win, tie, loss = simulate_match(hand=input_hand, trials=trials, opponents=opponents)

print(f'With hand: {input_hand}\nCategory: {hand_category(input_hand)}\nNumber of opponents: {opponents}\nSimulated trials: {trials}')
print(f'Win Probability: {win}\nTie Probability: {tie}\nLoss Probability: {loss}')

Input the suit of card 1 (H, D, S, or C) H
Input the value of card 1 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King) 1
Input the suit of card 2 (H, D, S, or C) S
Input the value of card 2 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King) 11
Input the suit of card 3 (H, D, S, or C) D
Input the value of card 3 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King) 9
Input the suit of card 4 (H, D, S, or C) S
Input the value of card 4 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King) 9
Input the suit of card 5 (H, D, S, or C) C
Input the value of card 5 (type a number; 1 for Ace, 11 for Jack, 12 for Queen, 13 for King) 9
How many trials would you like to run? 100
How many opponents do you have? 3


With hand: [['H', 1], ['S', 11], ['D', 9], ['S', 9], ['C', 9]]
Category: three-of-a-kind
Number of opponents: 3
Simulated trials: 100
Win Probability: 0.96
Tie Probability: 0.0
Loss Probability: 0.04
